# Adapted Attribute Type for quantities

**Purpose**: demonstrate using `dj.AttributeAdapter` for convenient storage of custom datatypes, e.g. objects from the quantities module.

Let's create a table with attribute `volume` storing values of type `mL` from the [`quantities`](https://pypi.org/project/quantities/) library.

In [1]:
import datajoint as dj
print(dj.__version__)

0.12.7


In [2]:
# The following is necessary in DataJoint version 0.12.* 
# while adapted types are in beta testing.
dj.errors._switch_adapted_types(True)  

In [3]:
import quantities as pq

class Milliliter(dj.AttributeAdapter):
    attribute_type = 'float'
        
    def put(self, obj: pq.Quantity) -> float:
        assert isinstance(obj, pq.Quantity)
        obj = obj.rescale(pq.mL)
        return obj.item()

    def get(self, value) -> str:
        return value * pq.mL

mL = Milliliter()

Then we need to define an adapter object that convert target objects into an attribute type that datajoint can already store. The class must subclass `dj.AttributeAdapter` and define the property `attribute_type`, and methods `get` and `put`. These methods translate the adapted data type `nx.Graph` into a representation that can be stored in datajoint, a `longblob` storing the edge list.

Now we can define a table that uses `graph` as its attribute type. These "adapted types" must be enclosed in angle brackets as in `<graph>`:

In [4]:
schema = dj.schema('test_quantities')
schema.drop()  # drop previous contents
schema = dj.schema('test_quantities')  # create de novo

Connecting dbadmin@dimitri-proj0.cda95qzjbnvs.us-east-1.rds.amazonaws.com:3306
Proceed to delete entire schema `test_quantities`? [yes, No]: yes


In [5]:
@schema
class Quantity(dj.Manual):
    definition = """
    quant_id : int
    ---
    volume : <mL>  
    """

In [6]:
Quantity.describe();

quant_id             : int                          
---
volume               : <mL>                         



### Now, populate the table with our example graphs and fetch them as objects
Inserting the graphs as objects

In [8]:
Quantity().insert([
    (1, 2.3 * pq.mL),
    (2, 3.5 * pq.mL)])

In [9]:
q = Quantity.fetch('volume')
q

array([array(2.3) * mL, array(3.5) * mL], dtype=object)

In [13]:
d = (3.5 * pq.mL).dtype

In [14]:
d

dtype('float64')